**Fire Prediction**

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

# Paths
TRAIN_DIR = "/content/drive/MyDrive/Data/Train_Data"
TEST_DIR  = "/content/drive/MyDrive/Data/Test_Data"

# Image parameters
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5  # You can increase for better accuracy

# 1️⃣ Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

print("Class indices:", train_generator.class_indices)

# 2️⃣ Model creation
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base model layers
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# 3️⃣ Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS
)

# 4️⃣ Save the model
model.save("/content/drive/MyDrive/fire_detection_mobilenetv2.h5")
print("Model saved!")

# 5️⃣ Load the model
model = load_model("/content/drive/MyDrive/fire_detection_mobilenetv2.h5")
print("Model loaded!")

# 6️⃣ Test images
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=1,
    class_mode='binary',
    shuffle=False
)

print("\nTesting Images ---")
for i in range(len(test_generator.filenames)):
    img_path = os.path.join(TEST_DIR, test_generator.filenames[i])
    img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    score = model.predict(img_array, verbose=0)[0][0]
    pred_label = "Fire" if score >= 0.5 else "Non-Fire"
    true_label = "Fire" if "Fire" in test_generator.filenames[i] else "Non-Fire"

    print(f"{test_generator.filenames[i]:<25} | True: {true_label:<8} | Score: {score:.3f} | Pred: {pred_label}")


Found 3689 images belonging to 2 classes.
Found 922 images belonging to 2 classes.
Class indices: {'Fire': 0, 'Non_Fire': 1}
Epoch 1/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.8303 - loss: 0.3924 - val_accuracy: 0.9360 - val_loss: 0.1910
Epoch 2/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 82s 704ms/step - accuracy: 0.9572 - loss: 0.1245 - val_accuracy: 0.9458 - val_loss: 0.1515
Epoch 3/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 81s 695ms/step - accuracy: 0.9642 - loss: 0.1082 - val_accuracy: 0.9447 - val_loss: 0.1589
Epoch 4/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 81s 700ms/step - accuracy: 0.9664 - loss: 0.0861 - val_accuracy: 0.9512 - val_loss: 0.1218
Epoch 5/5
116/116 ━━━━━━━━━━━━━━━━━━━━ 83s 713ms/step - accuracy: 0.9741 - loss: 0.0809 - val_accuracy: 0.9577 - val_loss: 0.1282


Model saved!


Model loaded!
Found 50 images belonging to 2 classes.

Testing Images ---
Fire/F_0.jpg              | True: Fire     | Score: 0.035 | Pred: Non-Fire
Fire/F_1.jpg              | True: Fire     | Score: 0.003 | Pred: Non-Fire
Fire/F_10.jpg             | True: Fire     | Score: 0.006 | Pred: Non-Fire
Fire/F_11.jpg             | True: Fire     | Score: 0.039 | Pred: Non-Fire
Fire/F_12.jpg             | True: Fire     | Score: 0.001 | Pred: Non-Fire
Fire/F_13.jpg             | True: Fire     | Score: 0.013 | Pred: Non-Fire
Fire/F_14.jpg             | True: Fire     | Score: 0.198 | Pred: Non-Fire
Fire/F_15.jpg             | True: Fire     | Score: 0.001 | Pred: Non-Fire
Fire/F_16.jpg             | True: Fire     | Score: 0.003 | Pred: Non-Fire
Fire/F_17.jpg             | True: Fire     | Score: 0.000 | Pred: Non-Fire
Fire/F_18.jpg             | True: Fire     | Score: 0.000 | Pred: Non-Fire
Fire/F_19.jpg             | True: Fire     | Score: 0.000 | Pred: Non-Fire
Fire/F_2.jpg              